In [ ]:
#Firstly select the correct graphs (100e or above) and get the data for it (in the correct form for the next function)
#Must enter in the correct order- smallest to largest

from ROOT import TCanvas, TGraph, TFile, TH1F, TH2D
import numpy as np
import re
import scipy as sc
import matplotlib.pyplot as plt
import array as arr

def collect_pulse_data(file):
    myFile = TFile.Open(file, "read")
    data = myFile.Get("PulseTransfer")
    detector = data.Get("detector1")
    pulsed_array = []
    data_x = []
    data_y = []
    for key in detector.GetListOfKeys():
        name = key.GetName()
        graph = detector.Get(name)
        if type(graph) != TGraph:
            continue
        else:
            searching = re.search("Accumulated", graph.GetTitle())
            searching1 = re.search(" = ", graph.GetTitle())
            if searching != None and name[:9] == "charge_ev":
                a = graph.GetTitle()[searching1.end():searching1.end()+4]
                searching_e = re.search("e", a)
                if searching_e != None:
                    a = a[:searching_e.end()-1]
                if np.abs(float(a)) >10:
                    a = float(a)/1000
                pulsed_array.append(a)
                event = detector.Get(name)
                x = event.GetX()
                y = event.GetY()
                data_x.append(x)
                data_y.append(y)
            
    return pulsed_array, data_x, data_y

#Code to firstly find the peaks of the fits for the pulses

def extract_data(x_data, y_data, pulse_names):
    acc_charge = arr.array('f', [])
    for x_pos, y_pos, ppos in zip(x_data, y_data, pulse_names):
        charge_z = arr.array('f', [])
        for x, y, p in zip(x_pos, y_pos, ppos):  
            if y == []:
                charge_point = 0
                charge_z.append(charge_point)
            elif float(p[0]) == 0:
                charge_point = 0
                charge_z.append(charge_point)
            else:
                grad = np.gradient(np.array(y[0]), np.array(x[0]))
                c = 0
                for i in y:
                    #c+=1
                    #if i>15 and c>66:
                    for a in range(len(x[0])):
                        print(np.array(x[0])[a])
                        if np.array(x[0])[a] >= 5 and np.array(x[0])[a-1] < 5:
                            idx = a
                            break
                        else:
                            continue
                    
                    #idx = np.where(grad==i)
                    charge_point = (np.array(y)[0][idx])*(1.6*(10**(-19)))
                    #print(charge_point)
                    charge_z.append(charge_point)
                    break
            
            #fig = plt.figure()
            #ax = fig.add_subplot(1,1,1)
            #ax.plot(x, y, label='Simulated Data')
            #plt.axvline(x= np.array(x)[0][idx][0])
            #ax.errorbar(np.abs(list(x))[:max_index-3], fit1, yerr=np.sqrt(np.diag(pcov1)[0]), color='purple')
            #ax.plot(x, y_fit, label='Fit for Simulated Data')
            #ax.set_title('Example Current Pulse for 10ps Laser and PIN Diode E_Field')
            #ax.set_ylabel('Current (\u03bcA)')
            #ax.set_xlabel('Time (ns)')
            #ax.legend()
            
        acc_charge = np.concatenate((acc_charge, charge_z))
        
    acc_charge = acc_charge.reshape((len(x_data), len(x_data[0])))
        
    return acc_charge

def plotting_depletion(acc_charge):
    
    fig, ax = plt.subplots()
    fig.set_size_inches(8, 5)
    im = ax.imshow(acc_charge, aspect='auto', extent= [-405, 405, -575, 575])
    ax.set_title('Depletion Depth of PIN Diode')
    #plt.xticks([-360, -330, -300, -270, -240, -210, -180, -150, -120, -90, -60, -30, 0, 30,   60, 90,  120,  150,  180,  210,  240,  270,  300,  330, 360,])
    #plt.yticks([-550,-500,-450,-400,-350,-300,-250,-200,-150,-100,-50,0,50,100,150,200,250,300,350,400,450,500,550])
    ax.set_xlabel('Position Along Depth into the Sensor (\u03bcm)')
    ax.set_ylabel('Position Across the top of the Sensor (\u03bcm)')
    fig.colorbar(im, ax=ax, label='Charge (C)')

    plt.show()

files = [['root_files/tcad_deplete_2_-550.0_-370.0.root', 'root_files/tcad_deplete_2_-550.0_-365.0.root', 'root_files/tcad_deplete_2_-550.0_-360.0.root', 'root_files/tcad_deplete_2_-550.0_-355.0.root', 'root_files/tcad_deplete_2_-550.0_-350.0.root', 'root_files/tcad_deplete_2_-550.0_-345.0.root', 'root_files/tcad_deplete_2_-550.0_-340.0.root', 'root_files/tcad_deplete_2_-550.0_-335.0.root', 'root_files/tcad_deplete_2_-550.0_-330.0.root', 'root_files/tcad_deplete_2_-550.0_-325.0.root', 'root_files/tcad_deplete_2_-550.0_-320.0.root', 'root_files/tcad_deplete_2_-550.0_-315.0.root', 'root_files/tcad_deplete_2_-550.0_-310.0.root', 'root_files/tcad_deplete_2_-550.0_-305.0.root', 'root_files/tcad_deplete_2_-550.0_-300.0.root', 'root_files/tcad_deplete_2_-550.0_-295.0.root', 'root_files/tcad_deplete_2_-550.0_-290.0.root', 'root_files/tcad_deplete_2_-550.0_-285.0.root', 'root_files/tcad_deplete_2_-550.0_-280.0.root', 'root_files/tcad_deplete_2_-550.0_-275.0.root', 'root_files/tcad_deplete_2_-550.0_-270.0.root', 'root_files/tcad_deplete_2_-550.0_-265.0.root', 'root_files/tcad_deplete_2_-550.0_-260.0.root', 'root_files/tcad_deplete_2_-550.0_-255.0.root', 'root_files/tcad_deplete_2_-550.0_-250.0.root'], ['root_files/tcad_deplete_2_-500.0_-370.0.root', 'root_files/tcad_deplete_2_-500.0_-365.0.root', 'root_files/tcad_deplete_2_-500.0_-360.0.root', 'root_files/tcad_deplete_2_-500.0_-355.0.root', 'root_files/tcad_deplete_2_-500.0_-350.0.root', 'root_files/tcad_deplete_2_-500.0_-345.0.root', 'root_files/tcad_deplete_2_-500.0_-340.0.root', 'root_files/tcad_deplete_2_-500.0_-335.0.root', 'root_files/tcad_deplete_2_-500.0_-330.0.root', 'root_files/tcad_deplete_2_-500.0_-325.0.root', 'root_files/tcad_deplete_2_-500.0_-320.0.root', 'root_files/tcad_deplete_2_-500.0_-315.0.root', 'root_files/tcad_deplete_2_-500.0_-310.0.root', 'root_files/tcad_deplete_2_-500.0_-305.0.root', 'root_files/tcad_deplete_2_-500.0_-300.0.root', 'root_files/tcad_deplete_2_-500.0_-295.0.root', 'root_files/tcad_deplete_2_-500.0_-290.0.root', 'root_files/tcad_deplete_2_-500.0_-285.0.root', 'root_files/tcad_deplete_2_-500.0_-280.0.root', 'root_files/tcad_deplete_2_-500.0_-275.0.root', 'root_files/tcad_deplete_2_-500.0_-270.0.root', 'root_files/tcad_deplete_2_-500.0_-265.0.root', 'root_files/tcad_deplete_2_-500.0_-260.0.root', 'root_files/tcad_deplete_2_-500.0_-255.0.root', 'root_files/tcad_deplete_2_-500.0_-250.0.root'], ['root_files/tcad_deplete_2_-450.0_-370.0.root', 'root_files/tcad_deplete_2_-450.0_-365.0.root', 'root_files/tcad_deplete_2_-450.0_-360.0.root', 'root_files/tcad_deplete_2_-450.0_-355.0.root', 'root_files/tcad_deplete_2_-450.0_-350.0.root', 'root_files/tcad_deplete_2_-450.0_-345.0.root', 'root_files/tcad_deplete_2_-450.0_-340.0.root', 'root_files/tcad_deplete_2_-450.0_-335.0.root', 'root_files/tcad_deplete_2_-450.0_-330.0.root', 'root_files/tcad_deplete_2_-450.0_-325.0.root', 'root_files/tcad_deplete_2_-450.0_-320.0.root', 'root_files/tcad_deplete_2_-450.0_-315.0.root', 'root_files/tcad_deplete_2_-450.0_-310.0.root', 'root_files/tcad_deplete_2_-450.0_-305.0.root', 'root_files/tcad_deplete_2_-450.0_-300.0.root', 'root_files/tcad_deplete_2_-450.0_-295.0.root', 'root_files/tcad_deplete_2_-450.0_-290.0.root', 'root_files/tcad_deplete_2_-450.0_-285.0.root', 'root_files/tcad_deplete_2_-450.0_-280.0.root', 'root_files/tcad_deplete_2_-450.0_-275.0.root', 'root_files/tcad_deplete_2_-450.0_-270.0.root', 'root_files/tcad_deplete_2_-450.0_-265.0.root', 'root_files/tcad_deplete_2_-450.0_-260.0.root', 'root_files/tcad_deplete_2_-450.0_-255.0.root', 'root_files/tcad_deplete_2_-450.0_-250.0.root'], ['root_files/tcad_deplete_2_-400.0_-370.0.root', 'root_files/tcad_deplete_2_-400.0_-365.0.root', 'root_files/tcad_deplete_2_-400.0_-360.0.root', 'root_files/tcad_deplete_2_-400.0_-355.0.root', 'root_files/tcad_deplete_2_-400.0_-350.0.root', 'root_files/tcad_deplete_2_-400.0_-345.0.root', 'root_files/tcad_deplete_2_-400.0_-340.0.root', 'root_files/tcad_deplete_2_-400.0_-335.0.root', 'root_files/tcad_deplete_2_-400.0_-330.0.root', 'root_files/tcad_deplete_2_-400.0_-325.0.root', 'root_files/tcad_deplete_2_-400.0_-320.0.root', 'root_files/tcad_deplete_2_-400.0_-315.0.root', 'root_files/tcad_deplete_2_-400.0_-310.0.root', 'root_files/tcad_deplete_2_-400.0_-305.0.root', 'root_files/tcad_deplete_2_-400.0_-300.0.root', 'root_files/tcad_deplete_2_-400.0_-295.0.root', 'root_files/tcad_deplete_2_-400.0_-290.0.root', 'root_files/tcad_deplete_2_-400.0_-285.0.root', 'root_files/tcad_deplete_2_-400.0_-280.0.root', 'root_files/tcad_deplete_2_-400.0_-275.0.root', 'root_files/tcad_deplete_2_-400.0_-270.0.root', 'root_files/tcad_deplete_2_-400.0_-265.0.root', 'root_files/tcad_deplete_2_-400.0_-260.0.root', 'root_files/tcad_deplete_2_-400.0_-255.0.root', 'root_files/tcad_deplete_2_-400.0_-250.0.root'], ['root_files/tcad_deplete_2_-350.0_-370.0.root', 'root_files/tcad_deplete_2_-350.0_-365.0.root', 'root_files/tcad_deplete_2_-350.0_-360.0.root', 'root_files/tcad_deplete_2_-350.0_-355.0.root', 'root_files/tcad_deplete_2_-350.0_-350.0.root', 'root_files/tcad_deplete_2_-350.0_-345.0.root', 'root_files/tcad_deplete_2_-350.0_-340.0.root', 'root_files/tcad_deplete_2_-350.0_-335.0.root', 'root_files/tcad_deplete_2_-350.0_-330.0.root', 'root_files/tcad_deplete_2_-350.0_-325.0.root', 'root_files/tcad_deplete_2_-350.0_-320.0.root', 'root_files/tcad_deplete_2_-350.0_-315.0.root', 'root_files/tcad_deplete_2_-350.0_-310.0.root', 'root_files/tcad_deplete_2_-350.0_-305.0.root', 'root_files/tcad_deplete_2_-350.0_-300.0.root', 'root_files/tcad_deplete_2_-350.0_-295.0.root', 'root_files/tcad_deplete_2_-350.0_-290.0.root', 'root_files/tcad_deplete_2_-350.0_-285.0.root', 'root_files/tcad_deplete_2_-350.0_-280.0.root', 'root_files/tcad_deplete_2_-350.0_-275.0.root', 'root_files/tcad_deplete_2_-350.0_-270.0.root', 'root_files/tcad_deplete_2_-350.0_-265.0.root', 'root_files/tcad_deplete_2_-350.0_-260.0.root', 'root_files/tcad_deplete_2_-350.0_-255.0.root', 'root_files/tcad_deplete_2_-350.0_-250.0.root'], ['root_files/tcad_deplete_2_-300.0_-370.0.root', 'root_files/tcad_deplete_2_-300.0_-365.0.root', 'root_files/tcad_deplete_2_-300.0_-360.0.root', 'root_files/tcad_deplete_2_-300.0_-355.0.root', 'root_files/tcad_deplete_2_-300.0_-350.0.root', 'root_files/tcad_deplete_2_-300.0_-345.0.root', 'root_files/tcad_deplete_2_-300.0_-340.0.root', 'root_files/tcad_deplete_2_-300.0_-335.0.root', 'root_files/tcad_deplete_2_-300.0_-330.0.root', 'root_files/tcad_deplete_2_-300.0_-325.0.root', 'root_files/tcad_deplete_2_-300.0_-320.0.root', 'root_files/tcad_deplete_2_-300.0_-315.0.root', 'root_files/tcad_deplete_2_-300.0_-310.0.root', 'root_files/tcad_deplete_2_-300.0_-305.0.root', 'root_files/tcad_deplete_2_-300.0_-300.0.root', 'root_files/tcad_deplete_2_-300.0_-295.0.root', 'root_files/tcad_deplete_2_-300.0_-290.0.root', 'root_files/tcad_deplete_2_-300.0_-285.0.root', 'root_files/tcad_deplete_2_-300.0_-280.0.root', 'root_files/tcad_deplete_2_-300.0_-275.0.root', 'root_files/tcad_deplete_2_-300.0_-270.0.root', 'root_files/tcad_deplete_2_-300.0_-265.0.root', 'root_files/tcad_deplete_2_-300.0_-260.0.root', 'root_files/tcad_deplete_2_-300.0_-255.0.root', 'root_files/tcad_deplete_2_-300.0_-250.0.root'], ['root_files/tcad_deplete_2_-250.0_-370.0.root', 'root_files/tcad_deplete_2_-250.0_-365.0.root', 'root_files/tcad_deplete_2_-250.0_-360.0.root', 'root_files/tcad_deplete_2_-250.0_-355.0.root', 'root_files/tcad_deplete_2_-250.0_-350.0.root', 'root_files/tcad_deplete_2_-250.0_-345.0.root', 'root_files/tcad_deplete_2_-250.0_-340.0.root', 'root_files/tcad_deplete_2_-250.0_-335.0.root', 'root_files/tcad_deplete_2_-250.0_-330.0.root', 'root_files/tcad_deplete_2_-250.0_-325.0.root', 'root_files/tcad_deplete_2_-250.0_-320.0.root', 'root_files/tcad_deplete_2_-250.0_-315.0.root', 'root_files/tcad_deplete_2_-250.0_-310.0.root', 'root_files/tcad_deplete_2_-250.0_-305.0.root', 'root_files/tcad_deplete_2_-250.0_-300.0.root', 'root_files/tcad_deplete_2_-250.0_-295.0.root', 'root_files/tcad_deplete_2_-250.0_-290.0.root', 'root_files/tcad_deplete_2_-250.0_-285.0.root', 'root_files/tcad_deplete_2_-250.0_-280.0.root', 'root_files/tcad_deplete_2_-250.0_-275.0.root', 'root_files/tcad_deplete_2_-250.0_-270.0.root', 'root_files/tcad_deplete_2_-250.0_-265.0.root', 'root_files/tcad_deplete_2_-250.0_-260.0.root', 'root_files/tcad_deplete_2_-250.0_-255.0.root', 'root_files/tcad_deplete_2_-250.0_-250.0.root'], ['root_files/tcad_deplete_2_-200.0_-370.0.root', 'root_files/tcad_deplete_2_-200.0_-365.0.root', 'root_files/tcad_deplete_2_-200.0_-360.0.root', 'root_files/tcad_deplete_2_-200.0_-355.0.root', 'root_files/tcad_deplete_2_-200.0_-350.0.root', 'root_files/tcad_deplete_2_-200.0_-345.0.root', 'root_files/tcad_deplete_2_-200.0_-340.0.root', 'root_files/tcad_deplete_2_-200.0_-335.0.root', 'root_files/tcad_deplete_2_-200.0_-330.0.root', 'root_files/tcad_deplete_2_-200.0_-325.0.root', 'root_files/tcad_deplete_2_-200.0_-320.0.root', 'root_files/tcad_deplete_2_-200.0_-315.0.root', 'root_files/tcad_deplete_2_-200.0_-310.0.root', 'root_files/tcad_deplete_2_-200.0_-305.0.root', 'root_files/tcad_deplete_2_-200.0_-300.0.root', 'root_files/tcad_deplete_2_-200.0_-295.0.root', 'root_files/tcad_deplete_2_-200.0_-290.0.root', 'root_files/tcad_deplete_2_-200.0_-285.0.root', 'root_files/tcad_deplete_2_-200.0_-280.0.root', 'root_files/tcad_deplete_2_-200.0_-275.0.root', 'root_files/tcad_deplete_2_-200.0_-270.0.root', 'root_files/tcad_deplete_2_-200.0_-265.0.root', 'root_files/tcad_deplete_2_-200.0_-260.0.root', 'root_files/tcad_deplete_2_-200.0_-255.0.root', 'root_files/tcad_deplete_2_-200.0_-250.0.root'], ['root_files/tcad_deplete_2_-150.0_-370.0.root', 'root_files/tcad_deplete_2_-150.0_-365.0.root', 'root_files/tcad_deplete_2_-150.0_-360.0.root', 'root_files/tcad_deplete_2_-150.0_-355.0.root', 'root_files/tcad_deplete_2_-150.0_-350.0.root', 'root_files/tcad_deplete_2_-150.0_-345.0.root', 'root_files/tcad_deplete_2_-150.0_-340.0.root', 'root_files/tcad_deplete_2_-150.0_-335.0.root', 'root_files/tcad_deplete_2_-150.0_-330.0.root', 'root_files/tcad_deplete_2_-150.0_-325.0.root', 'root_files/tcad_deplete_2_-150.0_-320.0.root', 'root_files/tcad_deplete_2_-150.0_-315.0.root', 'root_files/tcad_deplete_2_-150.0_-310.0.root', 'root_files/tcad_deplete_2_-150.0_-305.0.root', 'root_files/tcad_deplete_2_-150.0_-300.0.root', 'root_files/tcad_deplete_2_-150.0_-295.0.root', 'root_files/tcad_deplete_2_-150.0_-290.0.root', 'root_files/tcad_deplete_2_-150.0_-285.0.root', 'root_files/tcad_deplete_2_-150.0_-280.0.root', 'root_files/tcad_deplete_2_-150.0_-275.0.root', 'root_files/tcad_deplete_2_-150.0_-270.0.root', 'root_files/tcad_deplete_2_-150.0_-265.0.root', 'root_files/tcad_deplete_2_-150.0_-260.0.root', 'root_files/tcad_deplete_2_-150.0_-255.0.root', 'root_files/tcad_deplete_2_-150.0_-250.0.root'], ['root_files/tcad_deplete_2_-100.0_-370.0.root', 'root_files/tcad_deplete_2_-100.0_-365.0.root', 'root_files/tcad_deplete_2_-100.0_-360.0.root', 'root_files/tcad_deplete_2_-100.0_-355.0.root', 'root_files/tcad_deplete_2_-100.0_-350.0.root', 'root_files/tcad_deplete_2_-100.0_-345.0.root', 'root_files/tcad_deplete_2_-100.0_-340.0.root', 'root_files/tcad_deplete_2_-100.0_-335.0.root', 'root_files/tcad_deplete_2_-100.0_-330.0.root', 'root_files/tcad_deplete_2_-100.0_-325.0.root', 'root_files/tcad_deplete_2_-100.0_-320.0.root', 'root_files/tcad_deplete_2_-100.0_-315.0.root', 'root_files/tcad_deplete_2_-100.0_-310.0.root', 'root_files/tcad_deplete_2_-100.0_-305.0.root', 'root_files/tcad_deplete_2_-100.0_-300.0.root', 'root_files/tcad_deplete_2_-100.0_-295.0.root', 'root_files/tcad_deplete_2_-100.0_-290.0.root', 'root_files/tcad_deplete_2_-100.0_-285.0.root', 'root_files/tcad_deplete_2_-100.0_-280.0.root', 'root_files/tcad_deplete_2_-100.0_-275.0.root', 'root_files/tcad_deplete_2_-100.0_-270.0.root', 'root_files/tcad_deplete_2_-100.0_-265.0.root', 'root_files/tcad_deplete_2_-100.0_-260.0.root', 'root_files/tcad_deplete_2_-100.0_-255.0.root', 'root_files/tcad_deplete_2_-100.0_-250.0.root'], ['root_files/tcad_deplete_2_-50.0_-370.0.root', 'root_files/tcad_deplete_2_-50.0_-365.0.root', 'root_files/tcad_deplete_2_-50.0_-360.0.root', 'root_files/tcad_deplete_2_-50.0_-355.0.root', 'root_files/tcad_deplete_2_-50.0_-350.0.root', 'root_files/tcad_deplete_2_-50.0_-345.0.root', 'root_files/tcad_deplete_2_-50.0_-340.0.root', 'root_files/tcad_deplete_2_-50.0_-335.0.root', 'root_files/tcad_deplete_2_-50.0_-330.0.root', 'root_files/tcad_deplete_2_-50.0_-325.0.root', 'root_files/tcad_deplete_2_-50.0_-320.0.root', 'root_files/tcad_deplete_2_-50.0_-315.0.root', 'root_files/tcad_deplete_2_-50.0_-310.0.root', 'root_files/tcad_deplete_2_-50.0_-305.0.root', 'root_files/tcad_deplete_2_-50.0_-300.0.root', 'root_files/tcad_deplete_2_-50.0_-295.0.root', 'root_files/tcad_deplete_2_-50.0_-290.0.root', 'root_files/tcad_deplete_2_-50.0_-285.0.root', 'root_files/tcad_deplete_2_-50.0_-280.0.root', 'root_files/tcad_deplete_2_-50.0_-275.0.root', 'root_files/tcad_deplete_2_-50.0_-270.0.root', 'root_files/tcad_deplete_2_-50.0_-265.0.root', 'root_files/tcad_deplete_2_-50.0_-260.0.root', 'root_files/tcad_deplete_2_-50.0_-255.0.root', 'root_files/tcad_deplete_2_-50.0_-250.0.root'], ['root_files/tcad_deplete_2_0.0_-370.0.root', 'root_files/tcad_deplete_2_0.0_-365.0.root', 'root_files/tcad_deplete_2_0.0_-360.0.root', 'root_files/tcad_deplete_2_0.0_-355.0.root', 'root_files/tcad_deplete_2_0.0_-350.0.root', 'root_files/tcad_deplete_2_0.0_-345.0.root', 'root_files/tcad_deplete_2_0.0_-340.0.root', 'root_files/tcad_deplete_2_0.0_-335.0.root', 'root_files/tcad_deplete_2_0.0_-330.0.root', 'root_files/tcad_deplete_2_0.0_-325.0.root', 'root_files/tcad_deplete_2_0.0_-320.0.root', 'root_files/tcad_deplete_2_0.0_-315.0.root', 'root_files/tcad_deplete_2_0.0_-310.0.root', 'root_files/tcad_deplete_2_0.0_-305.0.root', 'root_files/tcad_deplete_2_0.0_-300.0.root', 'root_files/tcad_deplete_2_0.0_-295.0.root', 'root_files/tcad_deplete_2_0.0_-290.0.root', 'root_files/tcad_deplete_2_0.0_-285.0.root', 'root_files/tcad_deplete_2_0.0_-280.0.root', 'root_files/tcad_deplete_2_0.0_-275.0.root', 'root_files/tcad_deplete_2_0.0_-270.0.root', 'root_files/tcad_deplete_2_0.0_-265.0.root', 'root_files/tcad_deplete_2_0.0_-260.0.root', 'root_files/tcad_deplete_2_0.0_-255.0.root', 'root_files/tcad_deplete_2_0.0_-250.0.root'], ['root_files/tcad_deplete_2_50.0_-370.0.root', 'root_files/tcad_deplete_2_50.0_-365.0.root', 'root_files/tcad_deplete_2_50.0_-360.0.root', 'root_files/tcad_deplete_2_50.0_-355.0.root', 'root_files/tcad_deplete_2_50.0_-350.0.root', 'root_files/tcad_deplete_2_50.0_-345.0.root', 'root_files/tcad_deplete_2_50.0_-340.0.root', 'root_files/tcad_deplete_2_50.0_-335.0.root', 'root_files/tcad_deplete_2_50.0_-330.0.root', 'root_files/tcad_deplete_2_50.0_-325.0.root', 'root_files/tcad_deplete_2_50.0_-320.0.root', 'root_files/tcad_deplete_2_50.0_-315.0.root', 'root_files/tcad_deplete_2_50.0_-310.0.root', 'root_files/tcad_deplete_2_50.0_-305.0.root', 'root_files/tcad_deplete_2_50.0_-300.0.root', 'root_files/tcad_deplete_2_50.0_-295.0.root', 'root_files/tcad_deplete_2_50.0_-290.0.root', 'root_files/tcad_deplete_2_50.0_-285.0.root', 'root_files/tcad_deplete_2_50.0_-280.0.root', 'root_files/tcad_deplete_2_50.0_-275.0.root', 'root_files/tcad_deplete_2_50.0_-270.0.root', 'root_files/tcad_deplete_2_50.0_-265.0.root', 'root_files/tcad_deplete_2_50.0_-260.0.root', 'root_files/tcad_deplete_2_50.0_-255.0.root', 'root_files/tcad_deplete_2_50.0_-250.0.root'], ['root_files/tcad_deplete_2_100.0_-370.0.root', 'root_files/tcad_deplete_2_100.0_-365.0.root', 'root_files/tcad_deplete_2_100.0_-360.0.root', 'root_files/tcad_deplete_2_100.0_-355.0.root', 'root_files/tcad_deplete_2_100.0_-350.0.root', 'root_files/tcad_deplete_2_100.0_-345.0.root', 'root_files/tcad_deplete_2_100.0_-340.0.root', 'root_files/tcad_deplete_2_100.0_-335.0.root', 'root_files/tcad_deplete_2_100.0_-330.0.root', 'root_files/tcad_deplete_2_100.0_-325.0.root', 'root_files/tcad_deplete_2_100.0_-320.0.root', 'root_files/tcad_deplete_2_100.0_-315.0.root', 'root_files/tcad_deplete_2_100.0_-310.0.root', 'root_files/tcad_deplete_2_100.0_-305.0.root', 'root_files/tcad_deplete_2_100.0_-300.0.root', 'root_files/tcad_deplete_2_100.0_-295.0.root', 'root_files/tcad_deplete_2_100.0_-290.0.root', 'root_files/tcad_deplete_2_100.0_-285.0.root', 'root_files/tcad_deplete_2_100.0_-280.0.root', 'root_files/tcad_deplete_2_100.0_-275.0.root', 'root_files/tcad_deplete_2_100.0_-270.0.root', 'root_files/tcad_deplete_2_100.0_-265.0.root', 'root_files/tcad_deplete_2_100.0_-260.0.root', 'root_files/tcad_deplete_2_100.0_-255.0.root', 'root_files/tcad_deplete_2_100.0_-250.0.root'], ['root_files/tcad_deplete_2_150.0_-370.0.root', 'root_files/tcad_deplete_2_150.0_-365.0.root', 'root_files/tcad_deplete_2_150.0_-360.0.root', 'root_files/tcad_deplete_2_150.0_-355.0.root', 'root_files/tcad_deplete_2_150.0_-350.0.root', 'root_files/tcad_deplete_2_150.0_-345.0.root', 'root_files/tcad_deplete_2_150.0_-340.0.root', 'root_files/tcad_deplete_2_150.0_-335.0.root', 'root_files/tcad_deplete_2_150.0_-330.0.root', 'root_files/tcad_deplete_2_150.0_-325.0.root', 'root_files/tcad_deplete_2_150.0_-320.0.root', 'root_files/tcad_deplete_2_150.0_-315.0.root', 'root_files/tcad_deplete_2_150.0_-310.0.root', 'root_files/tcad_deplete_2_150.0_-305.0.root', 'root_files/tcad_deplete_2_150.0_-300.0.root', 'root_files/tcad_deplete_2_150.0_-295.0.root', 'root_files/tcad_deplete_2_150.0_-290.0.root', 'root_files/tcad_deplete_2_150.0_-285.0.root', 'root_files/tcad_deplete_2_150.0_-280.0.root', 'root_files/tcad_deplete_2_150.0_-275.0.root', 'root_files/tcad_deplete_2_150.0_-270.0.root', 'root_files/tcad_deplete_2_150.0_-265.0.root', 'root_files/tcad_deplete_2_150.0_-260.0.root', 'root_files/tcad_deplete_2_150.0_-255.0.root', 'root_files/tcad_deplete_2_150.0_-250.0.root'], ['root_files/tcad_deplete_2_200.0_-370.0.root', 'root_files/tcad_deplete_2_200.0_-365.0.root', 'root_files/tcad_deplete_2_200.0_-360.0.root', 'root_files/tcad_deplete_2_200.0_-355.0.root', 'root_files/tcad_deplete_2_200.0_-350.0.root', 'root_files/tcad_deplete_2_200.0_-345.0.root', 'root_files/tcad_deplete_2_200.0_-340.0.root', 'root_files/tcad_deplete_2_200.0_-335.0.root', 'root_files/tcad_deplete_2_200.0_-330.0.root', 'root_files/tcad_deplete_2_200.0_-325.0.root', 'root_files/tcad_deplete_2_200.0_-320.0.root', 'root_files/tcad_deplete_2_200.0_-315.0.root', 'root_files/tcad_deplete_2_200.0_-310.0.root', 'root_files/tcad_deplete_2_200.0_-305.0.root', 'root_files/tcad_deplete_2_200.0_-300.0.root', 'root_files/tcad_deplete_2_200.0_-295.0.root', 'root_files/tcad_deplete_2_200.0_-290.0.root', 'root_files/tcad_deplete_2_200.0_-285.0.root', 'root_files/tcad_deplete_2_200.0_-280.0.root', 'root_files/tcad_deplete_2_200.0_-275.0.root', 'root_files/tcad_deplete_2_200.0_-270.0.root', 'root_files/tcad_deplete_2_200.0_-265.0.root', 'root_files/tcad_deplete_2_200.0_-260.0.root', 'root_files/tcad_deplete_2_200.0_-255.0.root', 'root_files/tcad_deplete_2_200.0_-250.0.root'], ['root_files/tcad_deplete_2_250.0_-370.0.root', 'root_files/tcad_deplete_2_250.0_-365.0.root', 'root_files/tcad_deplete_2_250.0_-360.0.root', 'root_files/tcad_deplete_2_250.0_-355.0.root', 'root_files/tcad_deplete_2_250.0_-350.0.root', 'root_files/tcad_deplete_2_250.0_-345.0.root', 'root_files/tcad_deplete_2_250.0_-340.0.root', 'root_files/tcad_deplete_2_250.0_-335.0.root', 'root_files/tcad_deplete_2_250.0_-330.0.root', 'root_files/tcad_deplete_2_250.0_-325.0.root', 'root_files/tcad_deplete_2_250.0_-320.0.root', 'root_files/tcad_deplete_2_250.0_-315.0.root', 'root_files/tcad_deplete_2_250.0_-310.0.root', 'root_files/tcad_deplete_2_250.0_-305.0.root', 'root_files/tcad_deplete_2_250.0_-300.0.root', 'root_files/tcad_deplete_2_250.0_-295.0.root', 'root_files/tcad_deplete_2_250.0_-290.0.root', 'root_files/tcad_deplete_2_250.0_-285.0.root', 'root_files/tcad_deplete_2_250.0_-280.0.root', 'root_files/tcad_deplete_2_250.0_-275.0.root', 'root_files/tcad_deplete_2_250.0_-270.0.root', 'root_files/tcad_deplete_2_250.0_-265.0.root', 'root_files/tcad_deplete_2_250.0_-260.0.root', 'root_files/tcad_deplete_2_250.0_-255.0.root', 'root_files/tcad_deplete_2_250.0_-250.0.root'], ['root_files/tcad_deplete_2_300.0_-370.0.root', 'root_files/tcad_deplete_2_300.0_-365.0.root', 'root_files/tcad_deplete_2_300.0_-360.0.root', 'root_files/tcad_deplete_2_300.0_-355.0.root', 'root_files/tcad_deplete_2_300.0_-350.0.root', 'root_files/tcad_deplete_2_300.0_-345.0.root', 'root_files/tcad_deplete_2_300.0_-340.0.root', 'root_files/tcad_deplete_2_300.0_-335.0.root', 'root_files/tcad_deplete_2_300.0_-330.0.root', 'root_files/tcad_deplete_2_300.0_-325.0.root', 'root_files/tcad_deplete_2_300.0_-320.0.root', 'root_files/tcad_deplete_2_300.0_-315.0.root', 'root_files/tcad_deplete_2_300.0_-310.0.root', 'root_files/tcad_deplete_2_300.0_-305.0.root', 'root_files/tcad_deplete_2_300.0_-300.0.root', 'root_files/tcad_deplete_2_300.0_-295.0.root', 'root_files/tcad_deplete_2_300.0_-290.0.root', 'root_files/tcad_deplete_2_300.0_-285.0.root', 'root_files/tcad_deplete_2_300.0_-280.0.root', 'root_files/tcad_deplete_2_300.0_-275.0.root', 'root_files/tcad_deplete_2_300.0_-270.0.root', 'root_files/tcad_deplete_2_300.0_-265.0.root', 'root_files/tcad_deplete_2_300.0_-260.0.root', 'root_files/tcad_deplete_2_300.0_-255.0.root', 'root_files/tcad_deplete_2_300.0_-250.0.root'], ['root_files/tcad_deplete_2_350.0_-370.0.root', 'root_files/tcad_deplete_2_350.0_-365.0.root', 'root_files/tcad_deplete_2_350.0_-360.0.root', 'root_files/tcad_deplete_2_350.0_-355.0.root', 'root_files/tcad_deplete_2_350.0_-350.0.root', 'root_files/tcad_deplete_2_350.0_-345.0.root', 'root_files/tcad_deplete_2_350.0_-340.0.root', 'root_files/tcad_deplete_2_350.0_-335.0.root', 'root_files/tcad_deplete_2_350.0_-330.0.root', 'root_files/tcad_deplete_2_350.0_-325.0.root', 'root_files/tcad_deplete_2_350.0_-320.0.root', 'root_files/tcad_deplete_2_350.0_-315.0.root', 'root_files/tcad_deplete_2_350.0_-310.0.root', 'root_files/tcad_deplete_2_350.0_-305.0.root', 'root_files/tcad_deplete_2_350.0_-300.0.root', 'root_files/tcad_deplete_2_350.0_-295.0.root', 'root_files/tcad_deplete_2_350.0_-290.0.root', 'root_files/tcad_deplete_2_350.0_-285.0.root', 'root_files/tcad_deplete_2_350.0_-280.0.root', 'root_files/tcad_deplete_2_350.0_-275.0.root', 'root_files/tcad_deplete_2_350.0_-270.0.root', 'root_files/tcad_deplete_2_350.0_-265.0.root', 'root_files/tcad_deplete_2_350.0_-260.0.root', 'root_files/tcad_deplete_2_350.0_-255.0.root', 'root_files/tcad_deplete_2_350.0_-250.0.root'], ['root_files/tcad_deplete_2_400.0_-370.0.root', 'root_files/tcad_deplete_2_400.0_-365.0.root', 'root_files/tcad_deplete_2_400.0_-360.0.root', 'root_files/tcad_deplete_2_400.0_-355.0.root', 'root_files/tcad_deplete_2_400.0_-350.0.root', 'root_files/tcad_deplete_2_400.0_-345.0.root', 'root_files/tcad_deplete_2_400.0_-340.0.root', 'root_files/tcad_deplete_2_400.0_-335.0.root', 'root_files/tcad_deplete_2_400.0_-330.0.root', 'root_files/tcad_deplete_2_400.0_-325.0.root', 'root_files/tcad_deplete_2_400.0_-320.0.root', 'root_files/tcad_deplete_2_400.0_-315.0.root', 'root_files/tcad_deplete_2_400.0_-310.0.root', 'root_files/tcad_deplete_2_400.0_-305.0.root', 'root_files/tcad_deplete_2_400.0_-300.0.root', 'root_files/tcad_deplete_2_400.0_-295.0.root', 'root_files/tcad_deplete_2_400.0_-290.0.root', 'root_files/tcad_deplete_2_400.0_-285.0.root', 'root_files/tcad_deplete_2_400.0_-280.0.root', 'root_files/tcad_deplete_2_400.0_-275.0.root', 'root_files/tcad_deplete_2_400.0_-270.0.root', 'root_files/tcad_deplete_2_400.0_-265.0.root', 'root_files/tcad_deplete_2_400.0_-260.0.root', 'root_files/tcad_deplete_2_400.0_-255.0.root', 'root_files/tcad_deplete_2_400.0_-250.0.root'], ['root_files/tcad_deplete_2_450.0_-370.0.root', 'root_files/tcad_deplete_2_450.0_-365.0.root', 'root_files/tcad_deplete_2_450.0_-360.0.root', 'root_files/tcad_deplete_2_450.0_-355.0.root', 'root_files/tcad_deplete_2_450.0_-350.0.root', 'root_files/tcad_deplete_2_450.0_-345.0.root', 'root_files/tcad_deplete_2_450.0_-340.0.root', 'root_files/tcad_deplete_2_450.0_-335.0.root', 'root_files/tcad_deplete_2_450.0_-330.0.root', 'root_files/tcad_deplete_2_450.0_-325.0.root', 'root_files/tcad_deplete_2_450.0_-320.0.root', 'root_files/tcad_deplete_2_450.0_-315.0.root', 'root_files/tcad_deplete_2_450.0_-310.0.root', 'root_files/tcad_deplete_2_450.0_-305.0.root', 'root_files/tcad_deplete_2_450.0_-300.0.root', 'root_files/tcad_deplete_2_450.0_-295.0.root', 'root_files/tcad_deplete_2_450.0_-290.0.root', 'root_files/tcad_deplete_2_450.0_-285.0.root', 'root_files/tcad_deplete_2_450.0_-280.0.root', 'root_files/tcad_deplete_2_450.0_-275.0.root', 'root_files/tcad_deplete_2_450.0_-270.0.root', 'root_files/tcad_deplete_2_450.0_-265.0.root', 'root_files/tcad_deplete_2_450.0_-260.0.root', 'root_files/tcad_deplete_2_450.0_-255.0.root', 'root_files/tcad_deplete_2_450.0_-250.0.root'], ['root_files/tcad_deplete_2_500.0_-370.0.root', 'root_files/tcad_deplete_2_500.0_-365.0.root', 'root_files/tcad_deplete_2_500.0_-360.0.root', 'root_files/tcad_deplete_2_500.0_-355.0.root', 'root_files/tcad_deplete_2_500.0_-350.0.root', 'root_files/tcad_deplete_2_500.0_-345.0.root', 'root_files/tcad_deplete_2_500.0_-340.0.root', 'root_files/tcad_deplete_2_500.0_-335.0.root', 'root_files/tcad_deplete_2_500.0_-330.0.root', 'root_files/tcad_deplete_2_500.0_-325.0.root', 'root_files/tcad_deplete_2_500.0_-320.0.root', 'root_files/tcad_deplete_2_500.0_-315.0.root', 'root_files/tcad_deplete_2_500.0_-310.0.root', 'root_files/tcad_deplete_2_500.0_-305.0.root', 'root_files/tcad_deplete_2_500.0_-300.0.root', 'root_files/tcad_deplete_2_500.0_-295.0.root', 'root_files/tcad_deplete_2_500.0_-290.0.root', 'root_files/tcad_deplete_2_500.0_-285.0.root', 'root_files/tcad_deplete_2_500.0_-280.0.root', 'root_files/tcad_deplete_2_500.0_-275.0.root', 'root_files/tcad_deplete_2_500.0_-270.0.root', 'root_files/tcad_deplete_2_500.0_-265.0.root', 'root_files/tcad_deplete_2_500.0_-260.0.root', 'root_files/tcad_deplete_2_500.0_-255.0.root', 'root_files/tcad_deplete_2_500.0_-250.0.root'], ['root_files/tcad_deplete_2_550.0_-370.0.root', 'root_files/tcad_deplete_2_550.0_-365.0.root', 'root_files/tcad_deplete_2_550.0_-360.0.root', 'root_files/tcad_deplete_2_550.0_-355.0.root', 'root_files/tcad_deplete_2_550.0_-350.0.root', 'root_files/tcad_deplete_2_550.0_-345.0.root', 'root_files/tcad_deplete_2_550.0_-340.0.root', 'root_files/tcad_deplete_2_550.0_-335.0.root', 'root_files/tcad_deplete_2_550.0_-330.0.root', 'root_files/tcad_deplete_2_550.0_-325.0.root', 'root_files/tcad_deplete_2_550.0_-320.0.root', 'root_files/tcad_deplete_2_550.0_-315.0.root', 'root_files/tcad_deplete_2_550.0_-310.0.root', 'root_files/tcad_deplete_2_550.0_-305.0.root', 'root_files/tcad_deplete_2_550.0_-300.0.root', 'root_files/tcad_deplete_2_550.0_-295.0.root', 'root_files/tcad_deplete_2_550.0_-290.0.root', 'root_files/tcad_deplete_2_550.0_-285.0.root', 'root_files/tcad_deplete_2_550.0_-280.0.root', 'root_files/tcad_deplete_2_550.0_-275.0.root', 'root_files/tcad_deplete_2_550.0_-270.0.root', 'root_files/tcad_deplete_2_550.0_-265.0.root', 'root_files/tcad_deplete_2_550.0_-260.0.root', 'root_files/tcad_deplete_2_550.0_-255.0.root', 'root_files/tcad_deplete_2_550.0_-250.0.root']]

x_values, y_values, pulse_names = [], [], []
for dat in files:

    x, y, p = [], [], []
    for d in dat:
        pulse, x_dat, y_dat = collect_pulse_data(d)
        p.append(pulse)
        x.append(x_dat)
        y.append(y_dat)
        
    pulse_names.append(p)
    x_values.append(x)
    y_values.append(y)

acc_charge = extract_data(x_values, y_values, pulse_names)
print(acc_charge)
plotting_depletion(acc_charge)

In [ ]:
main_list = []
for aa in acc_charge:
    smaller = []
    for a in aa:
        smaller.append(a)
        if len(smaller) == 25:
            main_list.append(smaller)

print(main_list)  

In [ ]:
acc_charge_list4= np.array([[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, -1.4222222e-23, -6.4e-23, -1.6e-22, -2.791111e-22, -5.4222224e-22, -4.1244445e-22, -7.111111e-24, -4.2666665e-23, -3.431111e-22, -4.1066667e-22, -2.968889e-22, -1.6533333e-22, -4.4444445e-23, -8.888889e-24, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-1.4367399e-18, -1.542287e-17, -7.664498e-17, -1.8801942e-16, -2.6469515e-16, -3.109773e-16, -3.1790838e-16, -2.9053577e-16, -2.5205527e-16, -2.0247767e-16, -1.4776916e-16, -9.2494456e-17, -4.1725075e-17, -1.0384262e-17, -1.709333e-18, -2.3324443e-21, -1.01333336e-22, -3.5555554e-24, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-4.3195963e-18, -6.6251814e-17, -3.317211e-16, -6.1222604e-16, -6.8927396e-16, -6.965781e-16, -6.9896195e-16, -6.980813e-16, -6.951939e-16, -6.935525e-16, -6.910108e-16, -6.850017e-16, -6.666741e-16, -5.871285e-16, -3.599911e-16, -1.2616132e-16, -3.1029504e-17, -5.3904217e-18, -6.0730117e-19, -7.111111e-24, 0.0, 0.0, 0.0, 0.0, 0.0], [-4.3199735e-18, -6.5695505e-17, -3.2324935e-16, -5.9802866e-16, -6.8105605e-16, -6.934865e-16, -6.96634e-16, -6.9949124e-16, -6.990164e-16, -6.9770866e-16, -6.9682113e-16, -6.9297655e-16, -6.8102444e-16, -5.984336e-16, -3.4439028e-16, -1.060665e-16, -1.9098312e-17, -2.887173e-18, -3.2286969e-19, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-4.3200037e-18, -6.597161e-17, -3.2478618e-16, -6.028236e-16, -6.8605005e-16, -6.99263e-16, -7.002477e-16, -7.0084575e-16, -7.0059e-16, -6.9861636e-16, -6.9684464e-16, -6.935828e-16, -6.805604e-16, -5.94634e-16, -3.5077776e-16, -1.0934217e-16, -2.200293e-17, -3.2146979e-18, -3.4524428e-19, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-4.3200037e-18, -6.6014366e-17, -3.2556632e-16, -6.040757e-16, -6.861859e-16, -6.987112e-16, -7.007908e-16, -7.014739e-16, -6.991605e-16, -6.9994075e-16, -6.969296e-16, -6.945058e-16, -6.8155045e-16, -5.9593665e-16, -3.5093557e-16, -1.1452133e-16, -2.4349999e-17, -5.896942e-18, -8.845401e-19, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-4.3200037e-18, -6.6014644e-17, -3.260129e-16, -6.0390823e-16, -6.857557e-16, -6.990721e-16, -7.0089737e-16, -7.0063834e-16, -7.0096693e-16, -6.988992e-16, -6.9719674e-16, -6.9375376e-16, -6.798352e-16, -5.8991467e-16, -3.2935304e-16, -8.506534e-17, -9.752849e-18, -1.6291733e-19, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-4.3200037e-18, -6.6146556e-17, -3.2606756e-16, -6.0518847e-16, -6.8832486e-16, -6.9727366e-16, -7.0143815e-16, -7.012202e-16, -7.006949e-16, -6.985231e-16, -6.96032e-16, -6.951469e-16, -6.815237e-16, -5.867204e-16, -3.340996e-16, -9.388733e-17, -1.176432e-17, -5.0021566e-19, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-4.3200037e-18, -6.61465e-17, -3.26428e-16, -6.0339657e-16, -6.865718e-16, -6.98785e-16, -6.9890245e-16, -7.0028587e-16, -7.00708e-16, -6.9948976e-16, -6.9803895e-16, -6.940753e-16, -6.808847e-16, -5.8895006e-16, -3.2521946e-16, -8.195472e-17, -6.960687e-18, -1.6481244e-19, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-4.1600034e-18, -6.613512e-17, -3.2626867e-16, -6.034692e-16, -6.870222e-16, -7.001225e-16, -7.005796e-16, -7.004831e-16, -7.018305e-16, -6.983411e-16, -6.9676835e-16, -6.944159e-16, -6.8214565e-16, -5.9084826e-16, -3.306685e-16, -8.370488e-17, -1.04435655e-17, -4.422629e-19, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-4.1600034e-18, -6.5852504e-17, -3.2693182e-16, -6.035661e-16, -6.8740954e-16, -6.982544e-16, -7.0107064e-16, -7.0087556e-16, -7.0054193e-16, -6.984398e-16, -6.9732903e-16, -6.9458285e-16, -6.8181965e-16, -5.9092613e-16, -3.2901115e-16, -8.425037e-17, -8.464642e-18, -2.087319e-19, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-4.1600034e-18, -6.626434e-17, -3.2626867e-16, -6.0326597e-16, -6.864241e-16, -6.983417e-16, -6.9942486e-16, -7.0014976e-16, -7.0104385e-16, -6.987346e-16, -6.9660217e-16, -6.9452467e-16, -6.802836e-16, -5.895671e-16, -3.3247216e-16, -8.7796494e-17, -7.937928e-18, -3.064832e-19, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-4.3200037e-18, -6.613287e-17, -3.2653345e-16, -6.0313855e-16, -6.8773945e-16, -6.985239e-16, -7.008233e-16, -6.999573e-16, -7.006024e-16, -6.988061e-16, -6.9622244e-16, -6.951105e-16, -6.8214057e-16, -5.912269e-16, -3.327736e-16, -8.468463e-17, -7.92847e-18, -3.7188e-19, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-4.3200037e-18, -6.598387e-17, -3.2529768e-16, -6.022047e-16, -6.877024e-16, -6.9984377e-16, -6.992176e-16, -7.011233e-16, -7.003093e-16, -6.9851996e-16, -6.9808935e-16, -6.945768e-16, -6.806904e-16, -5.889807e-16, -3.3141167e-16, -9.256177e-17, -1.0706763e-17, -1.8224107e-19, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-4.3200037e-18, -6.6014644e-17, -3.253346e-16, -6.0371744e-16, -6.861292e-16, -6.986822e-16, -7.012121e-16, -7.011971e-16, -7.010752e-16, -6.982308e-16, -6.967747e-16, -6.947363e-16, -6.8168576e-16, -5.8611747e-16, -3.308499e-16, -8.996518e-17, -1.0178936e-17, -2.7253494e-19, -9.545262e-20, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-4.3200037e-18, -6.629783e-17, -3.2619533e-16, -6.0281667e-16, -6.868729e-16, -6.98858e-16, -7.003363e-16, -7.008632e-16, -7.0107673e-16, -6.9966467e-16, -6.973767e-16, -6.947441e-16, -6.8193437e-16, -5.946611e-16, -3.486842e-16, -1.1144034e-16, -2.2397968e-17, -5.1192864e-18, -8.2485013e-19, -3.9288888e-22, 0.0, 0.0, 0.0, 0.0, 0.0], [-4.3200037e-18, -6.64244e-17, -3.2560566e-16, -6.0171617e-16, -6.8772527e-16, -6.988407e-16, -7.01128e-16, -7.0035225e-16, -7.004514e-16, -6.9901256e-16, -6.9652017e-16, -6.945859e-16, -6.8092804e-16, -5.936411e-16, -3.4768037e-16, -1.0985791e-16, -2.37196e-17, -4.70485e-18, -1.2088889e-21, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-4.3199735e-18, -6.6168156e-17, -3.2520644e-16, -6.017277e-16, -6.8436413e-16, -6.958503e-16, -7.004772e-16, -6.996043e-16, -6.9852854e-16, -6.961933e-16, -6.9510636e-16, -6.9251164e-16, -6.803567e-16, -5.942692e-16, -3.4569776e-16, -9.465551e-17, -1.3476957e-17, -8.2930135e-19, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-4.319922e-18, -6.570097e-17, -3.225348e-16, -5.916756e-16, -6.766318e-16, -6.940954e-16, -6.9651784e-16, -6.970221e-16, -6.9573174e-16, -6.9290784e-16, -6.913107e-16, -6.8556226e-16, -6.688467e-16, -5.8504465e-16, -3.538362e-16, -1.2367744e-16, -2.9347968e-17, -5.5909303e-18, -5.549936e-19, -9.645067e-20, 0.0, 0.0, 0.0, 0.0, 0.0], [-1.8734367e-18, -1.8409943e-17, -8.52184e-17, -1.8332227e-16, -2.642282e-16, -3.043161e-16, -3.1358833e-16, -2.8936747e-16, -2.550884e-16, -1.9300435e-16, -1.2610538e-16, -7.400218e-17, -3.3378885e-17, -9.680388e-18, -1.2981028e-18, -1.0280889e-20, -1.1733333e-22, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 6.933333e-23, 3.0755556e-22, 8e-22, 7.217778e-22, 4.6755556e-22, 3.751111e-22, 9.653333e-22, 5.653333e-22, 1.1555555e-22, -2.5955556e-22, -2.968889e-22, -1.3866666e-22, -4.9777776e-23, -3.5555554e-24, -1.7777777e-24, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]
)
def dep(acc_charge_list1):
    max_vals = []
    for i in acc_charge_list1:
        max_vals.append(min(i))
    count_list = []
    for i in acc_charge_list1:
        count = 0
        for m,v in zip(max_vals, i):
            if np.abs(v) > (np.abs(m)/2):
                count+=1
        if count!=0:
            count_list.append(count)

    print(count_list)
    import statistics
    dep_d = statistics.mean(np.array(count_list))*5
    error = statistics.mean(np.sqrt(np.array(count_list))*5)
    print(f'{dep_d}\u00B1{error}')
    
def dep_lat(acc_charge_list1):
    charge_list = []
    max_vals = []
    for i in range(len(acc_charge_list1)):
        vals = []
        for c in acc_charge_list1:
            vals.append(c[i])
        charge_list.append(vals)
        
    for i in charge_list:   
        max_vals.append(min(i))
    count_list = []
    for i in acc_charge_list1:
        count = 0
        for m,v in zip(max_vals, i):
            if np.abs(v) > (np.abs(m)/2):
                count+=1
        if count!=0:
            count_list.append(count)

    print(count_list)
    import statistics
    dep_d = statistics.mean(np.array(count_list))*50
    error = statistics.mean(np.sqrt(np.array(count_list))*50)
    print(f'{dep_d}\u00B1{error}')

def plotting_depletion(acc_charge):
    
    fig, ax = plt.subplots()
    fig.set_size_inches(8, 5)
    im = ax.imshow(acc_charge, aspect='auto', extent= [247.5, 372.5, -575, 575], vmin=-8e-16)
    plt.axvline(x=362.5,color='red', ymin=0.045, ymax=0.955)
    plt.axhline(y=525,color='red', xmax=0.915)
    plt.axhline(y=-525,color='red', xmax=0.915)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    ax.set_xlabel('Position Along Depth of Sensor (\u03bcm)', fontsize=13)
    ax.set_ylabel('Position Along Pitch of Sensor (\u03bcm)', fontsize=13)
    cbar = plt.colorbar(im)
    cbar.set_label(label='Charge (C)', fontsize=13)
    cbar.ax.tick_params(labelsize=12)
    #fig.colorbar(im, ax=ax, label='Charge (C)')

    plt.show()
y = []
for i in acc_charge_list4:
    a = list(reversed(i))
    y.append(a)

plotting_depletion(y)
dep(acc_charge_list4)
dep_lat(acc_charge_list4)

In [ ]:
acc_charge_5 = np.array([[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, -1.2333689e-20, -6.444249e-20, -5.1659714e-19, -5.993367e-19, -7.5264373e-19, -6.1587185e-19, -4.8461333e-19, -1.7938809e-19, -7.5655645e-20, -4.7157334e-20, -3.099911e-20, -2.1592889e-20, -1.27911114e-20, -6.0426667e-21, -1.648e-21, -2.24e-22, -4.088889e-23, -3.5555554e-24, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-3.1734178e-18, -5.0543406e-17, -2.5178976e-16, -4.8902595e-16, -5.809907e-16, -6.1273823e-16, -6.3068263e-16, -6.3635526e-16, -6.0797976e-16, -5.006231e-16, -3.3222294e-16, -1.6009242e-16, -6.5430046e-17, -2.137268e-17, -3.7809983e-18, -7.3855664e-19, -1.4529778e-20, -2.4088889e-21, -1.9555556e-23, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-2.6492415e-18, -4.298354e-17, -2.1980655e-16, -4.3145783e-16, -5.385238e-16, -5.9608943e-16, -6.3916386e-16, -6.5230987e-16, -6.486725e-16, -6.0707905e-16, -5.0612e-16, -3.5078572e-16, -1.9728201e-16, -7.4332574e-17, -1.8566444e-17, -1.7622367e-18, -3.3969778e-20, -2.1546668e-21, -1.7777777e-24, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-2.5282103e-18, -4.1006576e-17, -2.1594721e-16, -4.2510638e-16, -5.35725e-16, -5.9825265e-16, -6.373445e-16, -6.526942e-16, -6.48963e-16, -6.1316625e-16, -5.1694405e-16, -3.7197084e-16, -2.0958721e-16, -8.23793e-17, -1.8797936e-17, -1.7821162e-18, -1.5004338e-19, -2.743822e-20, -1.7777777e-24, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-2.5279783e-18, -4.2565318e-17, -2.1581919e-16, -4.2728672e-16, -5.3629224e-16, -5.959914e-16, -6.3711976e-16, -6.517245e-16, -6.4788846e-16, -6.124568e-16, -5.1916476e-16, -3.751578e-16, -2.1267178e-16, -7.915655e-17, -1.7848053e-17, -1.8999857e-18, -1.380256e-19, -2.7235556e-21, -4.497778e-22, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-2.528061e-18, -4.2497585e-17, -2.1572271e-16, -4.2766654e-16, -5.350553e-16, -5.966758e-16, -6.361254e-16, -6.5181774e-16, -6.490849e-16, -6.141929e-16, -5.194961e-16, -3.7282454e-16, -2.1091296e-16, -8.1395075e-17, -2.0012646e-17, -2.1567256e-18, -6.4519286e-20, -8.4426666e-21, -1.7777777e-24, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-2.5277446e-18, -4.252139e-17, -2.1512624e-16, -4.2619757e-16, -5.34794e-16, -5.990235e-16, -6.3594175e-16, -6.523547e-16, -6.4699945e-16, -6.09132e-16, -5.1935195e-16, -3.7094341e-16, -2.1133117e-16, -8.2835296e-17, -1.8639676e-17, -2.101525e-18, -1.961984e-19, -3.8435557e-21, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-2.5277374e-18, -4.2492665e-17, -2.1479263e-16, -4.2809919e-16, -5.336523e-16, -5.966489e-16, -6.36649e-16, -6.5223607e-16, -6.4848197e-16, -6.1048724e-16, -5.1680424e-16, -3.7504912e-16, -2.125504e-16, -8.273062e-17, -1.8514277e-17, -2.225835e-18, -7.74992e-20, -2.6437511e-20, -4.9777776e-23, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-2.5279328e-18, -4.11261e-17, -2.1552503e-16, -4.2500775e-16, -5.359083e-16, -5.9580065e-16, -6.372914e-16, -6.522356e-16, -6.4867874e-16, -6.11101e-16, -5.146231e-16, -3.7446453e-16, -2.1261332e-16, -8.0690225e-17, -1.7967099e-17, -1.7143122e-18, -2.762878e-19, -1.3884445e-21, -2.8444444e-23, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-2.5648538e-18, -4.119378e-17, -2.1577516e-16, -4.240813e-16, -5.360357e-16, -5.976842e-16, -6.369311e-16, -6.530835e-16, -6.477569e-16, -6.120654e-16, -5.2024277e-16, -3.693397e-16, -2.1449157e-16, -8.208525e-17, -1.9136774e-17, -1.9592778e-18, -1.4279734e-19, -2.5882845e-20, -5.1555556e-23, -2.6666667e-23, 0.0, 0.0, 0.0, 0.0, 0.0], [-2.5279328e-18, -4.11261e-17, -2.1552503e-16, -4.2500775e-16, -5.3417963e-16, -5.9558015e-16, -6.385219e-16, -6.518969e-16, -6.472396e-16, -6.1276523e-16, -5.1383475e-16, -3.7338615e-16, -2.1046859e-16, -8.5581866e-17, -2.0575362e-17, -2.5464042e-18, -1.362336e-19, -2.2275556e-21, -1.7777777e-24, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-2.5268598e-18, -4.2552053e-17, -2.1507336e-16, -4.264173e-16, -5.347708e-16, -5.9709343e-16, -6.362769e-16, -6.5245185e-16, -6.4775103e-16, -6.127569e-16, -5.1668385e-16, -3.7638923e-16, -2.1581848e-16, -8.124323e-17, -1.9565538e-17, -2.69342e-18, -1.2875715e-19, -2.6572621e-20, -1.7777777e-24, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-2.5277382e-18, -4.2503114e-17, -2.1511512e-16, -4.26554e-16, -5.352451e-16, -5.9625127e-16, -6.361446e-16, -6.530774e-16, -6.473331e-16, -6.0892394e-16, -5.164514e-16, -3.669729e-16, -2.0936023e-16, -8.340785e-17, -1.8384713e-17, -2.2687132e-18, -2.289513e-19, -3.4595557e-21, -2.6666667e-23, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-2.528061e-18, -4.2497585e-17, -2.1572272e-16, -4.278267e-16, -5.350916e-16, -5.965652e-16, -6.376538e-16, -6.539062e-16, -6.503551e-16, -6.118465e-16, -5.220556e-16, -3.7549222e-16, -2.1105879e-16, -8.3621885e-17, -1.8854125e-17, -2.4179742e-18, -1.024656e-19, -3.0791112e-21, -2.8444444e-23, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-2.5279725e-18, -4.2533356e-17, -2.1504697e-16, -4.2830475e-16, -5.3509057e-16, -5.9645747e-16, -6.3634017e-16, -6.530572e-16, -6.4833845e-16, -6.1039666e-16, -5.187492e-16, -3.718312e-16, -2.0992217e-16, -8.6895145e-17, -1.7886904e-17, -1.8862942e-18, -1.2674826e-19, -2.864e-21, -1.7777777e-24, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-2.5281638e-18, -4.2529538e-17, -2.1593112e-16, -4.2664552e-16, -5.343286e-16, -5.982684e-16, -6.3640195e-16, -6.5236905e-16, -6.4765103e-16, -6.140772e-16, -5.1792634e-16, -3.6950176e-16, -2.1335575e-16, -8.289015e-17, -1.78836e-17, -2.2728568e-18, -1.717417e-19, -2.6037511e-20, -5.3333333e-23, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-2.6128714e-18, -4.2104608e-17, -2.179629e-16, -4.3179042e-16, -5.3934373e-16, -5.989053e-16, -6.3623387e-16, -6.5031055e-16, -6.4482914e-16, -6.0537863e-16, -5.0812734e-16, -3.479937e-16, -1.9960038e-16, -7.657706e-17, -1.6463199e-17, -1.6934344e-18, -4.1347557e-20, -3.3084444e-21, -2.6666667e-23, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-3.3676642e-18, -4.95061e-17, -2.4924672e-16, -4.776834e-16, -5.6940895e-16, -6.037785e-16, -6.266627e-16, -6.3343935e-16, -6.0380966e-16, -5.025203e-16, -3.2894194e-16, -1.6630179e-16, -6.898487e-17, -2.1009878e-17, -3.6585694e-18, -4.3073476e-19, -2.2444444e-20, -1.8062222e-21, -8.888889e-24, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, -1.2228622e-20, -1.6241031e-19, -3.3515502e-19, -7.7632513e-19, -6.1983823e-19, -5.787424e-19, -3.5978027e-19, -1.8735769e-19, -9.38288e-20, -5.1918224e-20, -3.0231112e-20, -2.2154666e-20, -1.3660445e-20, -6.0124443e-21, -1.5715555e-21, -2.8622221e-22, -3.5555554e-23, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]
)
y1 = []
for i in acc_charge_5:
    a = list(reversed(i))
    y1.append(a)

plotting_depletion(y1)

dep(acc_charge_5)
dep_lat(acc_charge_5)

In [ ]:
v50=[-4.3199735e-18, -6.5695505e-17, -3.2324935e-16, -5.9802866e-16, -6.8105605e-16, -6.934865e-16, -6.96634e-16, -6.9949124e-16, -6.990164e-16, -6.9770866e-16, -6.9682113e-16, -6.9297655e-16, -6.8102444e-16, -5.984336e-16, -3.4439028e-16, -1.060665e-16, -1.9098312e-17, -2.887173e-18, -3.2286969e-19, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
v2=[-2.6492415e-18, -4.298354e-17, -2.1980655e-16, -4.3145783e-16, -5.385238e-16, -5.9608943e-16, -6.3916386e-16, -6.5230987e-16, -6.486725e-16, -6.0707905e-16, -5.0612e-16, -3.5078572e-16, -1.9728201e-16, -7.4332574e-17, -1.8566444e-17, -1.7622367e-18, -3.3969778e-20, -2.1546668e-21, -1.7777777e-24, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.plot(np.linspace(370, 250, 25), v50, 'black', label='50V Charge Collection \nDepletion Depth: 60±17')
ax.plot(np.linspace(370, 250, 25), v2, 'blue', label='2V Charge Collection \nDepletion Depth: 50±15')
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
ax.set_ylabel('Accumulated Charge (C)', fontsize=13)
ax.set_xlabel('Position (\u03bcm)', fontsize=13)
ax.set_ylim(-1*10**(-15))
ax.legend(labelspacing = 1, fontsize=11)